In [1]:
#!pip install torch --quiet
#!pip install ray --quiet
#!pip install pydantic --quiet

## Model definition 

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split
from sklearn.metrics import matthews_corrcoef
import numpy as np
# Define the model
class SP_MLP(nn.Module): #eredita da nn.module alcune caratteristiche come la capacità di memorizzare i pesi
    def __init__(self, input_size, hidden_size1, hidden_size2, hidden_size3, output_size, dropout_p=0.5):
        super(SP_MLP, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size1) #ogni neurone in questo layer è connesso a tutti gli input. Indica quante feature riceve in input, per me questo sarà 39. e li trasformerà in 512 output.
        self.relu1 = nn.ReLU() #è una funzione di attivazione, se un numero in input è negativo lo trasforma in 0, altrimenti lo lascia invariato. Permette di disegnare pattern piu complessi di semplici combinazioni di regressioni lineari. Il fatto di trattare i dati negativi come 0 non è una perdita di dati, poichè 0 significa nessuna ricorrenza in quei dati, non ti interessa sapere "quanto non è presente il pattern in quel dato".
        self.dropout1 = nn.Dropout(p=dropout_p) #è una tecnica di regolarizzazione e previene l'overfitting, spegnendo casualmnente una percentuale (che in questo caso è dropout_p) dei neuroni di quel layer.
        self.fc2 = nn.Linear(hidden_size1, hidden_size2) #rifa le stesse cose di prima, prendendo i 512 output del layer precedente e trasformandoli in 256, e poi in 32 
        self.relu2 = nn.ReLU()
        self.dropout2 = nn.Dropout(p=dropout_p)
        self.fc3 = nn.Linear(hidden_size2, hidden_size3)
        self.relu3 = nn.ReLU()
        self.dropout3 = nn.Dropout(p=dropout_p)
        self.fc4 = nn.Linear(hidden_size3, output_size) #prende i 32 blocchi precedenti e li comprime in uno solo (output size)
        self.sigmoid = nn.Sigmoid() #schaiccia il valore prodotto da fc4 in un intervallo tra 0 e 1.

    def forward(self, x): #dice ai dati dove andare passo dopo passo ogni qualcvolta chiami il modello. Prende un batch di dati x e lo fa passare per ogni stazione/layer
        out = self.fc1(x)
        out = self.relu1(out)
        out = self.dropout1(out)
        out = self.fc2(out)
        out = self.relu2(out)
        out = self.dropout2(out)
        out = self.fc3(out)
        out = self.relu3(out)
        out = self.dropout3(out)
        out = self.fc4(out)
        out = self.sigmoid(out)
        return out

# Define a custom dataset
class HelixDataset(Dataset): #praticamente si tratta di tradurre i dati grezzi , ovvero le matrici numpy in formato che dataloader può capire e usare
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.float32) #converte la matrice x in un tensore di pytorch , ovvero la versione pytorch degli array numpy. I tensori sono oggetti particolari che possono essere spostati sulla GPU e essere usati per calcolare i gradienti.
        self.y = torch.tensor(y, dtype=torch.float32).reshape(-1, 1) #trasforma la matrice y di forma (4809,) in (4809,1) per renderla compatibile con l'output del modello che adotta questa forma.
    def __len__(self):
        return len(self.X) #risponde alla domanda "quanti campioni ci sono nel dataset?" e usa questo numero per sapere quanti batch può creare quando un'epoca è finita (ovvero quando ha guardato tutti i dati , ovvero ha terminato un epoca). Semplicemente divide la lunghezza totale del dataset per la dimensione del batch, per calcolarsi quanti batch completi corrispondono a un epoca. In ogni modo questa funzione fornsice la lunghezza del dataset per questo calcolo.

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx] #raccoglie la i-esima riga nella matrice x e y, a ogni chiamata del dataloader (quando è in modalità shuffle, e quindi li pesca a caso è molto importante, perche grazie a questa funzione lui va a pescare il numero chiamato dallo shuffle, per riga di campione e classe).


#def train_val è la funzione che gestisce l'intero processo di addestramento e validazione. 
def train_val(model, #è il modello da addestrare
              train_loader, #i dati da studiare, diviso in batch
              val_loader, #il test da fare a fine  di ogni studio
              optimizer, #metodo di studio (adam, RMSprop ecc.. dice al modlelo come aggiornare le sue consocenze).
              criterion, #il correttore, che dice al modello di quanto ha sbagliato
              epochs, #quante volte il modello rileggerà i dati per impararne
              patience, #quante volte il modello può fare un esame di prova peggiore del precedente prima di interrompere le epoche in anticipo
              scorer = matthews_corrcoef,
              init_best_score = -1,
              output_transform = lambda x: (x > 0.5).float()): #come tradurre la probabilità del modello, praticamente trasforma i valori in 1 e 0
  best_val_score = init_best_score #inizializza il miglior punteggio
  epochs_without_improvement = 0 #contatore della patience utile per vedere quante volte di fila non migliora
  best_model_state_dict = None #prepara il cassetto dove inserirci il modello che ha performato meglio

  for epoch in range(epochs): #ripeti il processo per epoche volte. 
      # Training
      model.train()  #inizializzi il modello vuoto da allenare
      loss = 0 #inizializzi la variabile per la loss
      for batch_X, batch_y in train_loader: #questo for itera su tutti i batches
          batch_X, batch_y = batch_X.to(device), batch_y.to(device) #sposta eventualmente i dati del batch sulla gpu se disponibile per fare i calcoli piu velocemente
          optimizer.zero_grad() #azzera l'optimizer che era stato utilizzato per il batch precedente
          outputs = model(batch_X) #il modello legge il batch x e produce le risposte
          loss = criterion(outputs, batch_y) # il correttore calcola il singolo numero di errore confrontando le risposte date dal modello con quelle del batch y
          loss.backward() #funzione di pytorch che  si guarda quanto ogni peso ha contribuito a quell'errore  tramite il calcolo del gradiente quindi dice di quanto un peso deve scendere o salire.
          optimizer.step() #prende i calcoli della backward e aggiorna fisicamente i pesi del cervello per ridurre l'errore

      # Validation
      model.eval() #è cruciale perche mette il modello in fase di valutazione, spegnendo il dropout, ovvero quello che spegneva neuroni a caso per evitare overfitting
      val_preds = []
      val_labels = []
      with torch.no_grad(): #dice a pytorch di non calcolare gradienti, poiche siamo in fase di valutazione, rendendo il tutto piu veloce e consumando meno memoria
          for batch_X, batch_y in val_loader: #itera su tutti i batch del validation
              batch_X, batch_y = batch_X.to(device), batch_y.to(device)
              outputs = model(batch_X)
              #preds = (outputs > 0.5).float() #qui invece utilizzi direttamente questo modo per trasformare gli output in 0 e 1
              preds = output_transform(outputs) #utilizza il metodo di traformazione conenuto in output _transform permettendolo di variare a piacimento
              val_preds.extend(preds.cpu().numpy().flatten()) #aggiunge le risposte alle liste
              val_labels.extend(batch_y.cpu().numpy().flatten())
      val_score = scorer(val_labels, val_preds) #calcola il punteggio MCC alla fine di ogni test

      if val_score > best_val_score:
          best_val_score = val_score
          epochs_without_improvement = 0
          best_model_state_dict = model.state_dict()
          print('Validation score improved to {:.4f}'.format(best_val_score))
      else:
          epochs_without_improvement += 1
          if epochs_without_improvement >= patience:
              print('Early stopping at epoch {}'.format(epoch+1))
              break

      print('Epoch [{}/{}], Loss: {:.4f}, Val score: {:.4f}'.format(epoch+1, epochs, loss.item(), val_score))
  return best_model_state_dict

def test(model, test_loader, scorer = matthews_corrcoef, output_transform = lambda x: (x > 0.5).float()):
  model.eval()
  all_preds = []
  all_labels = []
  with torch.no_grad():
      for batch_X, batch_y in test_loader:
          batch_X, batch_y = batch_X.to(device), batch_y.to(device)
          outputs = model(batch_X)
          preds = output_transform(outputs)
          all_preds.extend(preds.cpu().numpy().flatten())
          all_labels.extend(batch_y.cpu().numpy().flatten())

  score = scorer(all_labels, all_preds)
  return score


if torch.cuda.is_available():
    device = torch.device("cuda")
    print("GPU is available")
else:
    device = torch.device("cpu")
    print("GPU is not available")

GPU is not available


## Finding the best Hyperparameter

In [3]:
from ray import tune
from ray.tune import CLIReporter
from ray.tune.schedulers import ASHAScheduler
import os

In [4]:
#set the configuration on the random search must work
config = {
    "base_dir": os.path.abspath("../Feature_Selection/"),
    "h1": tune.choice([256, 512]), #hidden layer 1 dimension
    "h2": tune.choice([128, 256]), #hidden layer 2 dimension
    "h3": tune.choice([32, 64]), #hidden layer 3 dimension
    "dropout": tune.uniform(0.1, 0.5), #dropout percentage
    "lr": tune.loguniform(1e-4, 1e-2), #learning rate
    "batch_size": tune.choice([64, 128, 256]) #batch size
}

In [5]:
#this function makes the 5-CV to evaluate the mean mcc for that configuration of hyperparameter
def test_config(config):
    # Esempio: usa i tuoi 5 split .npz per fare MCC medio
    mcc_scores = []
    base_dir = config["base_dir"]
    for i in range(1, 6):
        #this line define the folder where npz(s) files are contained and the respective path
        train_path = os.path.join(base_dir, f"training_features_{i}.npz")
        val_path   = os.path.join(base_dir, f"validation_features_{i}.npz")
        test_path  = os.path.join(base_dir, f"testing_features_{i}.npz")
    # load feature matrices and label vector
        #load train
        loaded_data_train = np.load(train_path)
        x_train = loaded_data_train['matrix']
        y_train = loaded_data_train['target']
        
        # load test
        loaded_data_test = np.load(test_path)
        x_test = loaded_data_test['matrix']
        y_test = loaded_data_test['target']
        
        # load validation
        loaded_data_validation = np.load(val_path)
        x_val = loaded_data_validation['matrix']
        y_val = loaded_data_validation['target']
    
            # Split the dataset into training, validation and test sets
        train_dataset = HelixDataset(x_train, y_train)
        val_dataset = HelixDataset(x_val, y_val)
        test_dataset = HelixDataset(x_test, y_test)

                # Create data loaders divided in batches
        batch_size = config["batch_size"]
        train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
        val_loader = DataLoader(val_dataset, batch_size=batch_size)
        test_loader = DataLoader(test_dataset, batch_size=batch_size)
        
        # Initialize the model
        input_size = x_train.shape[1]
        
        model = SP_MLP(
            input_size,
            config["h1"], config["h2"], config["h3"],
            output_size=1,
            dropout_p=config["dropout"]).to(device)

        optimizer = optim.Adam(model.parameters(), lr=config["lr"]) #use adam optimized with learning rate chosen by random search
        criterion = nn.BCELoss()  #loss function for binary classification

        best_state = train_val(model, train_loader, val_loader,
                               optimizer, criterion,
                               epochs=100, patience=20)
        model.load_state_dict(best_state) #best state(with optimized weight) with chosen parameter 
        
        # calcoli MCC sul test di quel fold 
        mcc = test(model, test_loader)
        mcc_scores.append(mcc)

    mean_mcc = np.mean(mcc_scores)

    # raytune vuole che gli riporti un dizionario di metriche
    tune.report({"mcc": mean_mcc, "loss": -mean_mcc})


In [6]:
scheduler = ASHAScheduler(
    metric="mcc", #which metric use to evaluate which trials are good or not
    mode="max", #which method to use (for example for mcc we want that it maximize, for loss indeed minimize)
    max_t=100,   # number of max epoche
    grace_period=1, #minimum number of epoche that the each trial must do befor be judge
    reduction_factor=2 #halving factor, it says how much trial must be terminated at each round of ASHA. In our case it is indifferent because we have only one round of ASHA
)

result = tune.run(
    test_config,
    config=config,
    num_samples=20,      # how many combinations you try 
    scheduler=scheduler
)

best_trial = result.get_best_trial("mcc", "max", "last") #search all trials (combination of configurations) and select the best one. It compares the last reported mcc from each trial and returns the trial that achieved the maximum mcc.
print("Best trial config:", best_trial.config) #take the best configuration
print("Best CV MCC:", best_trial.last_result["mcc"]) #take the best mcc


2025-11-10 19:33:20,775	INFO worker.py:2012 -- Started a local Ray instance.
/home/markus/anaconda3/envs/lab1/lib/python3.13/site-packages/ray/_private/worker.py:2051: FutureWarning: Tip: In future versions of Ray, Ray will no longer override accelerator visible devices env var if num_gpus=0 or num_gpus=None (default). To enable this behavior and turn off this error message, set RAY_ACCEL_ENV_VAR_OVERRIDE_ON_ZERO=0
  warnings.warn(
2025-11-10 19:33:21,908	INFO tune.py:253 -- Initializing Ray automatically. For cluster usage or custom Ray initialization, call `ray.init(...)` before `tune.run(...)`.
2025-11-10 19:33:21,912	INFO tune.py:616 -- [output] This uses the legacy output and progress reporter, as Jupyter notebooks are not supported by the new engine, yet. For more information, please see https://github.com/ray-project/ray/issues/36949
2025-11-10 19:33:21,944	INFO tensorboardx.py:193 -- pip install "ray[tune]" to see TensorBoard files.
2025-11-10 19:33:21,945	WARNING callback.py:1

(test_config pid=13994) Validation score improved to 0.6763
(test_config pid=13994) Epoch [1/100], Loss: 0.3172, Val score: 0.6763
(test_config pid=13999) Validation score improved to 0.7515 [repeated 31x across cluster] (Ray deduplicates logs by default. Set RAY_DEDUP_LOGS=0 to disable log deduplication, or see https://docs.ray.io/en/master/ray-observability/user-guides/configure-logging.html#log-deduplication for more options.)
(test_config pid=13994) Epoch [19/100], Loss: 0.1288, Val score: 0.7149 [repeated 76x across cluster]
(test_config pid=13994) Validation score improved to 0.7601 [repeated 17x across cluster]
(test_config pid=13994) Epoch [42/100], Loss: 0.2109, Val score: 0.7490 [repeated 94x across cluster]
(test_config pid=13994) Validation score improved to 0.7667 [repeated 9x across cluster]
(test_config pid=13994) Epoch [60/100], Loss: 0.0559, Val score: 0.7516 [repeated 74x across cluster]


(pid=gcs_server) [2025-11-10 19:33:48,706 E 13673 13673] (gcs_server) gcs_server.cc:302: Failed to establish connection to the event+metrics exporter agent. Events and metrics will not be exported. Exporter agent status: RpcError: Running out of retries to initialize the metrics agent. rpc_code: 14


(test_config pid=13996) Validation score improved to 0.8019 [repeated 8x across cluster]
(test_config pid=14000) Epoch [97/100], Loss: 0.1848, Val score: 0.7193 [repeated 75x across cluster]


(raylet) [2025-11-10 19:33:50,764 E 13748 13748] (raylet) main.cc:975: Failed to establish connection to the metrics exporter agent. Metrics will not be exported. Exporter agent status: RpcError: Running out of retries to initialize the metrics agent. rpc_code: 14
[2025-11-10 19:33:51,918 E 13644 13809] core_worker_process.cc:825: Failed to establish connection to the metrics exporter agent. Metrics will not be exported. Exporter agent status: RpcError: Running out of retries to initialize the metrics agent. rpc_code: 14
(test_config pid=13994) [2025-11-10 19:33:53,561 E 13994 14057] core_worker_process.cc:825: Failed to establish connection to the metrics exporter agent. Metrics will not be exported. Exporter agent status: RpcError: Running out of retries to initialize the metrics agent. rpc_code: 14


...

(test_config pid=15029) Early stopping at epoch 61
(test_config pid=15352) Validation score improved to 0.7368 [repeated 10x across cluster]
(test_config pid=15352) Epoch [96/100], Loss: 0.1758, Val score: 0.7368 [repeated 57x across cluster]
(test_config pid=15352) Validation score improved to 0.7628 [repeated 14x across cluster]
(test_config pid=15185) Epoch [6/100], Loss: 0.1181, Val score: 0.7005 [repeated 50x across cluster]
(test_config pid=15352) Early stopping at epoch 32
(test_config pid=15185) Validation score improved to 0.7567 [repeated 18x across cluster]
(test_config pid=15433) Epoch [31/100], Loss: 0.0897, Val score: 0.7632 [repeated 94x across cluster]
(test_config pid=15271) Early stopping at epoch 57
(test_config pid=15433) Validation score improved to 0.8064 [repeated 18x across cluster]
(test_config pid=15352) Epoch [18/100], Loss: 0.1843, Val score: 0.7183 [repeated 79x across cluster]
(test_config pid=15185) Validation score improved to 0.7927 [repeated 14x across

(test_config pid=15433) [2025-11-10 19:43:12,906 E 15433 15459] core_worker_process.cc:825: Failed to establish connection to the metrics exporter agent. Metrics will not be exported. Exporter agent status: RpcError: Running out of retries to initialize the metrics agent. rpc_code: 14


(test_config pid=15433) Validation score improved to 0.8177 [repeated 7x across cluster]
(test_config pid=15433) Epoch [55/100], Loss: 0.0708, Val score: 0.8095 [repeated 62x across cluster]
(test_config pid=15352) Validation score improved to 0.7645 [repeated 5x across cluster]
(test_config pid=15352) Epoch [86/100], Loss: 0.1976, Val score: 0.7280 [repeated 62x across cluster]
(test_config pid=15433) Early stopping at epoch 88
(test_config pid=15433) Validation score improved to 0.8075 [repeated 29x across cluster]
(test_config pid=15352) Epoch [9/100], Loss: 0.1323, Val score: 0.6610 [repeated 80x across cluster]
(test_config pid=15271) Early stopping at epoch 57
(test_config pid=15433) Early stopping at epoch 66
(test_config pid=15433) Validation score improved to 0.6888 [repeated 14x across cluster]
(test_config pid=15271) Epoch [28/100], Loss: 0.0979, Val score: 0.6823 [repeated 63x across cluster]
(test_config pid=15352) Early stopping at epoch 44
(test_config pid=15352) Validat

2025-11-10 19:44:09,761	INFO tune.py:1009 -- Wrote the latest version of all result files and experiment state to '/home/markus/ray_results/test_config_2025-11-10_19-33-21' in 0.0158s.
2025-11-10 19:44:09,789	INFO tune.py:1041 -- Total run time: 647.88 seconds (647.78 seconds for the tuning loop).


Best trial config: {'base_dir': '/mnt/c/Users/chari/OneDrive - Alma Mater Studiorum Università di Bologna/Scuola/Università/MAGISTRALE/GitHub/LB2_project_Group_3/Feature_Selection', 'h1': 256, 'h2': 128, 'h3': 32, 'dropout': 0.12171227302487551, 'lr': 0.0007498655247946051, 'batch_size': 128}
Best CV MCC: 0.7987457359908747


## Benchmark set and final evaluation